### Analysing the emails of enron employees for descrepencies with keywords "bankrupt", "fraud", "shutdown". Finding out the people who used these between below mentioned dates. Also analyse on which months theses word were used mostly.<br>

Key dates:<br>
- August 2001, Jeffrey Skilling resigned as CEO of Enron. 
- December 2, 2001 - Enron files for Chapter 11 protection, becoming the largest bankruptcy in U.S. history.

In [35]:
import nltk, os, glob, email, datetime, re
import email.utils
import time
import datetime

directory = 'data/enron/enron_mail_20150507/maildir/'

In [41]:
user_list = []

name_email_map = dict()
bankrupt = dict()
fraud = dict()
shutdown = dict()

bankrupt_date = dict()
fraud_date = dict()
shutdown_date = dict()

#identifying list of users
for users in os.walk(directory):
    user_list = users[1]
    break
#print(user_list)   

print("Traversing through each user folders...")
for user in user_list:
    print(user)
    user_folder = directory + user
    private_server_mails = dict()

    #identifying sub directories under user folder
    for d in os.walk(user_folder):
        sub_dir = d[1]
        break

    # identifying folders with word sent in it
    sent_folders = []
    if sub_dir:
        for x in sub_dir:
            if x.startswith('sent'):
                sent_folders.append(x)

    #iterating through sent folders
    #print('Iterating through sent folders')
    for sent_folder in sent_folders:
        sent_folder_files = user_folder +'/'+sent_folder+'/*'
        root, dirs, sent_folder_filenames = next(os.walk(user_folder +'/'+sent_folder))

        for filename in sent_folder_filenames:
            fname = user_folder +'/'+sent_folder+'/'+filename
            with open(fname,'r',encoding='utf-8',errors='ignore') as file:
                #whole email
                message = email.message_from_string(file.read())

                #keys
                #print(message.keys())

                #from
                if 'From' in message.keys():
                    from_email = message.get('From')
                if 'X-From' in message.keys():
                    from_name = message.get('X-From')

                #print(from_email)
                #storing sender email in map
                if from_email not in name_email_map:
                    name_email_map[from_email] = from_name

                #date
                #initial analysis shows date in PDT/PST
                sdate = message.get('Date')
                # Fri, 16 Feb 2001 09:26:00 -0800 (PST)
                #datetime_object = datetime.datetime.strptime(sdate, '%b %d %Y %I:%M%p')
                #print(sdate.datetime())
                pdate = email.utils.parsedate(sdate)
                make_time = time.mktime(pdate)
                parse_date = datetime.datetime.fromtimestamp(make_time)
                #print(parse_date) 1999-02-16 06:39:00
                #Checking for month entry in dictionary
                date_entry = parse_date.strftime("%B")+'-'+parse_date.strftime("%Y")
                
                #body
                if not message.is_multipart():
                    body = message.get_payload()
                #print(body)

                #cleaning email body using regex
                body_list = re.sub('[^a-zA-Z0-9 \n\.]', '', body)
                body_list = (body_list.replace('.','')).split()
                #print(body_list)

                for word in body_list:
                    if word.lower() == 'bankrupt':
                        #if bankrupt word present in body of email
                        if from_email in bankrupt.keys():
                            bankrupt[from_email] = bankrupt[from_email] + 1
                        else:
                            bankrupt[from_email] = 1
                        
                        if date_entry in bankrupt_date.keys():
                            bankrupt_date[date_entry] = bankrupt_date[date_entry] + 1
                        else:
                            bankrupt_date[date_entry] = 1
                    elif word.lower() == 'fraud':
                        #if fraud word present in body of email
                        if from_email in fraud.keys():
                            fraud[from_email] = fraud[from_email] + 1
                        else:
                            fraud[from_email] = 1
                        
                        if date_entry in fraud_date.keys():
                            fraud_date[date_entry] = fraud_date[date_entry] + 1
                        else:
                            fraud_date[date_entry] = 1
                    elif word.lower() == 'shutdown':
                        #if shutdown word present in body of email
                        if from_email in shutdown.keys():
                            shutdown[from_email] = shutdown[from_email] + 1
                        else:
                            shutdown[from_email] = 1
                        
                        if date_entry in shutdown_date.keys():
                            shutdown_date[date_entry] = shutdown_date[date_entry] + 1
                        else:
                            shutdown_date[date_entry] = 1


                            
print("\n'Bankrupt' word analysis.")    
print("Keyword most used people.")
sorted_bankrupt = [(k, bankrupt[k]) for k in sorted(bankrupt, key=bankrupt.get, reverse=True)]
for b in sorted_bankrupt[:5]:
    print(str(b[1])+' - '+b[0])
#print(sorted_bankrupt)
print("Jeff Dasovich, Enron governmental affairs executive")
print("Steven Kean, Enron’s former chief of staff")

print("\n'Bankrupt' most used months.")
sorted_bankrupt_date = [(k, bankrupt_date[k]) for k in sorted(bankrupt_date, key=bankrupt_date.get, reverse=True)]
#print(sorted_bankrupt_date)
for b in sorted_bankrupt_date[:5]:
    print(str(b[1])+' - '+b[0])
print("April 2001 - 3 months before Jeffrey Skilling resigns as CEO, and Kenneth Lay becomes CEO again")  
print("November 2001 - Dynegy announces it has terminated merger talks with Enron.")
        
  

    
print("\n'Fraud' word analysis.")    
print("Keyword most used people.")
sorted_fraud = [(k, fraud[k]) for k in sorted(fraud, key=fraud.get, reverse=True)]
for b in sorted_fraud[:5]:
    print(str(b[1])+' - '+b[0])
#print(sorted_fraud)
print("Richard Sanders, Enron's assistant general counsel")
print("Mark Haedicke received hefty bonuses before bankruptcy : $750,000")

print("\n'Fraud' most used months.")
sorted_fraud_date = [(k, fraud_date[k]) for k in sorted(fraud_date, key=fraud_date.get, reverse=True)]
#print(sorted_fraud_date)
for b in sorted_fraud_date[:5]:
    print(str(b[1])+' - '+b[0])
print("November 2001 - Dynegy announces merger and termnation of merger talks with Enron.")
print("October 2001 - The SEC opens a formal investigation into Enron's transactions.")    
    
    
print("\n'Shutdown' word analysis.")    
print("Keyword most used people.")
sorted_shutdown = [(k, shutdown[k]) for k in sorted(shutdown, key=shutdown.get, reverse=True)]
for b in sorted_shutdown[:5]:
    print(str(b[1])+' - '+b[0])
#print(sorted_shutdown)
print("Mike S. McConnell, previous President of Enron Global Markets, LLC")

print("\n'Shutdown' most used months.")
sorted_shutdown_date = [(k, shutdown_date[k]) for k in sorted(shutdown_date, key=shutdown_date.get, reverse=True)]
#print(sorted_shutdown_date)
for b in sorted_shutdown_date[:5]:
    print(str(b[1])+' - '+b[0])
print("June 2000 the peiod when Enron stock value rise to record high of US$90.75 per share.")

Traversing through each user folders...
allen-p
arnold-j
arora-h
badeer-r
bailey-s
bass-e
baughman-d
beck-s
benson-r
blair-l
brawner-s
buy-r
campbell-l
carson-m
cash-m
causholli-m
corman-s
crandell-s
cuilla-m
dasovich-j
davis-d
dean-c
delainey-d
derrick-j
dickson-s
donoho-l
donohoe-t
dorland-c
ermis-f
farmer-d
fischer-m
forney-j
fossum-d
gang-l
gay-r
geaccone-t
germany-c
gilbertsmith-d
giron-d
griffith-j
grigsby-m
guzman-m
haedicke-m
hain-m
harris-s
hayslett-r
heard-m
hendrickson-s
hernandez-j
hodge-j
holst-k
horton-s
hyatt-k
hyvl-d
jones-t
kaminski-v
kean-s
keavey-p
keiser-k
king-j
kitchen-l
kuykendall-t
lavorato-j
lay-k
lenhart-m
lewis-a
linder-e
lokay-m
lokey-t
love-p
lucci-p
maggi-m
mann-k
martin-t
may-l
mccarty-d
mcconnell-m
mckay-b
mckay-j
mclaughlin-e
merriss-s
meyers-a
mims-thurston-p
motley-m
neal-s
nemec-g
panus-s
parks-j
pereira-s
perlingiere-d
phanis-s
pimenov-v
platter-p
presto-k
quenet-j
quigley-d
rapp-b
reitmeyer-j
richey-c
ring-a
ring-r
rodrique-r
rogers-b
ruscitti-k
sa